In [43]:
from langchain_ollama import ChatOllama
from amadeus import Client, ResponseError
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.agents import initialize_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.agents import Tool



In [44]:
llm = ChatOllama(
    model="llama3-groq-tool-use",
    temperature=0,
)

In [45]:


class FlightSearchSchema(BaseModel):
    originLocationCode: str = Field(..., description="First airport code")
    destinationLocationCode: str = Field(..., description="Second airport code")
    departureDate: str = Field(..., description="Date of departure")
    adults: int = Field(..., description="Number of adults")

@tool
def flight_search(query: str):
    """
    Searches flights based on the provided parameters.
    The query should include origin, destination, departure date, and number of adults.
    """
    # Parse the query string into individual parameters
    params = {}
    for param in query.split(','):
        key, value = param.strip().split(':')
        params[key.strip()] = value.strip()
    
    try:
        # Validate and create FlightSearchSchema instance
        search_params = FlightSearchSchema(
            originLocationCode=params['originLocationCode'],
            destinationLocationCode=params['destinationLocationCode'],
            departureDate=params['departureDate'],
            adults=int(params['adults'])
        )
        
        amadeus = Client(
            client_id='5G1CNwDfyBXnZuKLoAkicMoqgEqn26Ex',
            client_secret='0yDZezKgpgNP215B'
        )
        
        response = amadeus.shopping.flight_offers_search.get(
            originLocationCode=search_params.originLocationCode,
            destinationLocationCode=search_params.destinationLocationCode,
            departureDate=search_params.departureDate,
            adults=search_params.adults
        )
        return str(response.data)  # Convert to string for easier handling
    except KeyError as e:
        return f"Missing parameter: {str(e)}"
    except ValueError as e:
        return f"Invalid parameter value: {str(e)}"
    except ResponseError as error:
        return f"API error: {error.response.body}"

In [46]:
flight_search = Tool(
    name='flight_search',
    func=flight_search,
    description="""
    Searches for flights based on provided parameters. 
    Input should be a string with comma-separated key-value pairs: 
    'originLocationCode: XXX, destinationLocationCode: YYY, departureDate: YYYY-MM-DD, adults: N'
    Convert city names to airport codes before using. 
    If any required information is missing, ask the user for it.
    """
)

In [47]:

tools = [flight_search]

# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)


# create our agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

In [48]:
conversational_agent("give me flights from nyc to bom.give me top 5 flights on 25th august, 2024")



> Entering new AgentExecutor chain...
```json
{
    "action": "flight_search",
    "action_input": "originLocationCode: JFK, destinationLocationCode: BOM, departureDate: 2024-08-25, adults: 1"
}
```
Observation: [{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'isUpsellOffer': False, 'lastTicketingDate': '2024-08-19', 'lastTicketingDateTime': '2024-08-19', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT18H25M', 'segments': [{'departure': {'iataCode': 'JFK', 'terminal': '4', 'at': '2024-08-25T15:30:00'}, 'arrival': {'iataCode': 'AUH', 'terminal': 'A', 'at': '2024-08-26T12:30:00'}, 'carrierCode': 'EY', 'number': '102', 'aircraft': {'code': '388'}, 'operating': {'carrierCode': 'EY'}, 'duration': 'PT13H', 'id': '59', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'AUH', 'terminal': 'A', 'at': '2024-08-26T14:40:00'}, 'arrival': {'iataCode': 'BOM', 'terminal': '2', 'at'